<a href="https://colab.research.google.com/github/nananana25/MSC151/blob/main/COIL_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install libs
!pip install yfinance xgboost shap --quiet

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

In [ ]:
start = "2021-01-01"
end   = "2024-06-30"

tickers = {
    "GBPIDR": "GBPIDR=X",   # GBP/IDR FX cross (Yahoo)
    "GOLD":   "GC=F",       # Gold futures (proxy for gold spot)
    "BRENT":  "BZ=F",       # Brent crude futures
    "FTSE":   "^FTSE",      # UK stock index (FTSE 100)
    "JCI":    "^JKSE"       # Indonesia stock index (Jakarta Composite Index) - Yahoo sometimes uses ^JKSE
}

# Download historical daily close prices
data = {}
for name, ticker in tickers.items():
    print("Downloading", ticker)
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty:
        print("WARNING: no data for", ticker)
    series = df["Close"]
    series.name = name
    data[name] = series

# Combine into single DataFrame
df_daily = pd.concat(data.values(), axis=1)
df_daily.index.name = "Date"
df_daily = df_daily.sort_index()
df_daily.head()

/tmp/ipython-input-1379909918.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)


/tmp/ipython-input-1379909918.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)
/tmp/ipython-input-1379909918.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)
/tmp/ipython-input-1379909918.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)


/tmp/ipython-input-1379909918.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)


Ticker,GBPIDR=X,GC=F,BZ=F,^FTSE,^JKSE
Date,,,,,
2021-01-01,18855.154297,NaN,NaN,NaN,NaN
2021-01-04,19007.109375,1944.699951,51.090000,6571.899902,6104.897949
2021-01-05,18829.888672,1952.699951,53.599998,6612.299805,6137.342773
2021-01-06,18936.498047,1906.900024,54.299999,6841.899902,6065.682129
2021-01-07,18855.943359,1912.300049,54.380001,6857.000000,6153.632812


**Reasoning**:
The previous code executed successfully, combining the data into `df_daily` with correct Series names. Now, I need to verify its structure by checking its information (data types, non-null counts) to ensure it's ready for further processing.



In [ ]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 911 entries, 2021-01-01 to 2024-06-28
Freq: B
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GBPIDR=X  911 non-null    float64
 1   GC=F      877 non-null    float64
 2   BZ=F      878 non-null    float64
 3   ^FTSE     879 non-null    float64
 4   ^JKSE     842 non-null    float64
dtypes: float64(5)
memory usage: 42.7 KB
